In [1]:
import os

In [2]:
%pwd

'd:\\Text-Summarizer-Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Text-Summarizer-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name:Path

In [6]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_dir

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath= PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_dir([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_dir([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )    

        return data_transformation_config

In [8]:
%pip install transformers
%pip install datasets
%pip install sentencepiece
%pip install --upgrade transformers
%pip install protobuf

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
from TextSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

c:\Users\preet\anaconda3\envs\TextS\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [14]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained("google/pegasus-cnn_dailymail")

    def convert_examples_to_features(self, examples):
        input_encodings = self.tokenizer(examples['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(examples['summary'], max_length = 128, truncation = True )

        return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt = dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [15]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    data_transformation.convert()

except Exception as e:
    raise e

[2023-09-28 12:07:28,153: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-28 12:07:28,154: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-28 12:07:28,156: INFO: common: created directory at :artifacts]
[2023-09-28 12:07:28,158: INFO: common: created directory at :artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\preet\anaconda3\envs\TextS\Lib\site-packages\transformers\tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 154574.73 examples/s]
